<a href="https://colab.research.google.com/github/chansu0613/Paper-works/blob/main/DANN_OfficeHome_dataset_Domain_Adaptation_%EC%8B%A4%ED%97%98.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 드라이브 마운트

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
%load_ext autoreload
%autoreload 2


In [ ]:
%cd /content/gdrive/MyDrive/2023-1 연구폴더/
!pwd
!ls

[Errno 2] No such file or directory: '/content/gdrive/MyDrive/2023-1 연구폴더/'
/content
/content
gdrive	sample_data


In [ ]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models
import gdown
import zipfile

# 데이터셋 압축 해제 코드(실행 금지)

In [ ]:
# Office-Home 데이터셋 다운로드
# zip_url = 'https://drive.google.com/uc?id=0B81rNlvomiwed0V1YUxQdC1uOTg'
output = '/content/gdrive/MyDrive/OfficeHomeDataset_10072016.zip'
# gdown.download(url, output, quiet=False)

dataset_path = '/content/gdrive/MyDrive/OfficeHomeDataset'

# 대상 디렉토리가 없으면 생성
os.makedirs(dataset_path, exist_ok=True)

# 압축 해제
!unzip -q '/content/gdrive/MyDrive/OfficeHomeDataset_10072016.zip' -d '/content/gdrive/MyDrive/OfficeHomeDataset'

# 데이터셋 정의 코드

In [ ]:
# from tensorflow.keras.preprocessing.sequence import pad_sequences

# # 데이터 로드 및 전처리 함수
# def load_and_preprocess_data(domain_path, is_source=True):
#     images = []
#     labels = []
#     for class_name in os.listdir(domain_path):
#         class_path = os.path.join(domain_path, class_name)
#         if os.path.isdir(class_path):
#             for img_name in os.listdir(class_path):
#                 img_path = os.path.join(class_path, img_name)
#                 img = tf.keras.preprocessing.image.load_img(img_path, target_size=(224, 224))
#                 img_array = tf.keras.preprocessing.image.img_to_array(img)
#                 images.append(img_array)
#                 labels.append(class_name)

#     images = np.array(images) / 255.0
#     labels = np.array(labels)

#     if is_source:
#         label_encoder = tf.keras.preprocessing.text.Tokenizer()
#         label_encoder.fit_on_texts(labels)
#         encoded_labels = label_encoder.texts_to_sequences(labels)
#         encoded_labels = np.array([label[0] for label in encoded_labels]) # Extract the single label value
#         return images, encoded_labels, label_encoder
#     else:
#         return images, labels

# # 소스 도메인(Art)과 타겟 도메인(Clipart) 데이터 로드
# source_path = '/content/gdrive/MyDrive/OfficeHomeDataset/OfficeHomeDataset_10072016/Art'
# target_path = '/content/gdrive/MyDrive/OfficeHomeDataset/OfficeHomeDataset_10072016/Clipart'

# source_images, source_labels, label_encoder = load_and_preprocess_data(source_path)
# target_images, target_labels = load_and_preprocess_data(target_path, is_source=False)

# # 데이터셋 생성
# BATCH_SIZE = 32
# source_dataset = tf.data.Dataset.from_tensor_slices((source_images, source_labels)).shuffle(1000).batch(BATCH_SIZE)
# target_dataset = tf.data.Dataset.from_tensor_slices(target_images).shuffle(1000).batch(BATCH_SIZE)

In [ ]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.preprocessing.text import Tokenizer

def image_generator(domain_path, batch_size=32, target_size=(224, 224)):
    class_names = [d for d in os.listdir(domain_path) if os.path.isdir(os.path.join(domain_path, d))]
    image_paths = []
    labels = []

    for class_name in class_names:
        class_path = os.path.join(domain_path, class_name)
        for img_name in os.listdir(class_path):
            image_paths.append(os.path.join(class_path, img_name))
            labels.append(class_name)

    num_samples = len(image_paths)
    while True:
        indices = np.random.permutation(num_samples)
        for start in range(0, num_samples, batch_size):
            end = min(start + batch_size, num_samples)
            batch_indices = indices[start:end]

            batch_images = []
            batch_labels = []
            for i in batch_indices:
                img = load_img(image_paths[i], target_size=target_size)
                img_array = img_to_array(img)
                batch_images.append(img_array)
                batch_labels.append(labels[i])

            yield np.array(batch_images) / 255.0, np.array(batch_labels)

def create_datasets(source_path, target_path, batch_size=32):
    source_gen = image_generator(source_path, batch_size)
    target_gen = image_generator(target_path, batch_size)

    # 레이블 인코더 생성
    all_labels = []
    for _, labels in source_gen:
        all_labels.extend(labels)
        if len(all_labels) >= 1000:  # 충분한 레이블 샘플을 얻으면 중단
            break

    label_encoder = Tokenizer()
    label_encoder.fit_on_texts(all_labels)

    def encode_labels(gen):
        for images, labels in gen:
            encoded_labels = label_encoder.texts_to_sequences(labels)
            # encoded_labels = np.array([label[0] for label in encoded_labels])
            encoded_labels = np.array([label[0] -1 for label in encoded_labels]) # Subtract 1 from each label
            yield images, encoded_labels

    source_dataset = tf.data.Dataset.from_generator(
        lambda: encode_labels(source_gen),
        output_signature=(
            tf.TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32),
            tf.TensorSpec(shape=(None,), dtype=tf.int32)
        )
    )

    target_dataset = tf.data.Dataset.from_generator(
        lambda: (images for images, _ in target_gen),
        output_signature=tf.TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32)
    )

    return source_dataset, target_dataset, label_encoder

# 데이터셋 생성
BATCH_SIZE = 32
source_path = '/content/gdrive/MyDrive/OfficeHomeDataset/OfficeHomeDataset_10072016/Art'
target_path = '/content/gdrive/MyDrive/OfficeHomeDataset/OfficeHomeDataset_10072016/Clipart'

source_dataset, target_dataset, label_encoder = create_datasets(source_path, target_path, BATCH_SIZE)

KeyboardInterrupt: 

In [ ]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.preprocessing.text import Tokenizer
from tqdm import tqdm
def image_generator(domain_path, batch_size=32, target_size=(224, 224)):
    class_names = [d for d in os.listdir(domain_path) if os.path.isdir(os.path.join(domain_path, d))]
    image_paths = []
    labels = []

    print(f"Loading image paths from {domain_path}...")
    for class_name in tqdm(class_names, desc="Classes"):
        class_path = os.path.join(domain_path, class_name)
        for img_name in os.listdir(class_path):
            image_paths.append(os.path.join(class_path, img_name))
            labels.append(class_name)

    num_samples = len(image_paths)
    print(f"Total images found: {num_samples}")

    while True:
        indices = np.random.permutation(num_samples)
        for start in tqdm(range(0, num_samples, batch_size), desc="Batches", leave=False):
            end = min(start + batch_size, num_samples)
            batch_indices = indices[start:end]

            batch_images = []
            batch_labels = []
            for i in batch_indices:
                img = load_img(image_paths[i], target_size=target_size)
                img_array = img_to_array(img)
                batch_images.append(img_array)
                batch_labels.append(labels[i])

            yield np.array(batch_images) / 255.0, np.array(batch_labels)

        print("Completed one full pass through the dataset.")

def create_datasets(source_path, target_path, batch_size=32):
    source_gen = image_generator(source_path, batch_size)
    target_gen = image_generator(target_path, batch_size)

    # 레이블 인코더 생성
    all_labels = []
    for _, labels in source_gen:
        all_labels.extend(labels)
        if len(all_labels) >= 1000:  # 충분한 레이블 샘플을 얻으면 중단
            break

    label_encoder = Tokenizer()
    label_encoder.fit_on_texts(all_labels)

    def encode_labels(gen):
        for images, labels in gen:
            encoded_labels = label_encoder.texts_to_sequences(labels)
            # encoded_labels = np.array([label[0] for label in encoded_labels])
            encoded_labels = np.array([label[0] -1 for label in encoded_labels]) # Subtract 1 from each label
            yield images, encoded_labels

    source_dataset = tf.data.Dataset.from_generator(
        lambda: encode_labels(source_gen),
        output_signature=(
            tf.TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32),
            tf.TensorSpec(shape=(None,), dtype=tf.int32)
        )
    )

    target_dataset = tf.data.Dataset.from_generator(
        lambda: (images for images, _ in target_gen),
        output_signature=tf.TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32)
    )

    return source_dataset, target_dataset, label_encoder

# 데이터셋 생성
BATCH_SIZE = 32
source_path = '/content/gdrive/MyDrive/OfficeHomeDataset/OfficeHomeDataset_10072016/Art'
target_path = '/content/gdrive/MyDrive/OfficeHomeDataset/OfficeHomeDataset_10072016/Clipart'

source_dataset, target_dataset, label_encoder = create_datasets(source_path, target_path, BATCH_SIZE)

Loading image paths from /content/gdrive/MyDrive/OfficeHomeDataset/OfficeHomeDataset_10072016/Art...


Classes: 100%|██████████| 65/65 [00:00<00:00, 1326.30it/s]


Total images found: 2427


Batches:  41%|████      | 31/76 [10:36<14:59, 19.98s/it]

In [ ]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.preprocessing.text import Tokenizer

def get_all_labels(domain_path):
    class_names = [d for d in os.listdir(domain_path) if os.path.isdir(os.path.join(domain_path, d))]
    return class_names

def image_generator(domain_path, label_encoder, target_size=(224, 224)):
    class_names = [d for d in os.listdir(domain_path) if os.path.isdir(os.path.join(domain_path, d))]
    image_paths = []
    labels = []

    for class_name in class_names:
        class_path = os.path.join(domain_path, class_name)
        for img_name in os.listdir(class_path):
            image_paths.append(os.path.join(class_path, img_name))
            labels.append(class_name)

    num_samples = len(image_paths)
    indices = np.arange(num_samples)
    np.random.shuffle(indices)

    for i in indices:
        img = load_img(image_paths[i], target_size=target_size)
        img_array = img_to_array(img)
        encoded_label = label_encoder.texts_to_sequences([labels[i]])[0][0] - 1
        yield np.array(img_array) / 255.0, encoded_label

def create_datasets(source_path, target_path):
    # 모든 레이블 가져오기
    all_labels = get_all_labels(source_path)

    # 레이블 인코더 생성 및 학습
    label_encoder = Tokenizer()
    label_encoder.fit_on_texts(all_labels)

    source_dataset = tf.data.Dataset.from_generator(
        lambda: image_generator(source_path, label_encoder),
        output_signature=(
            tf.TensorSpec(shape=(224, 224, 3), dtype=tf.float32),
            tf.TensorSpec(shape=(), dtype=tf.int32)
        )
    )

    target_dataset = tf.data.Dataset.from_generator(
        lambda: (image for image, _ in image_generator(target_path, label_encoder)),
        output_signature=tf.TensorSpec(shape=(224, 224, 3), dtype=tf.float32)
    )

    return source_dataset, target_dataset, label_encoder

# 데이터셋 생성
source_path = '/content/gdrive/MyDrive/OfficeHomeDataset/OfficeHomeDataset_10072016/Art'
target_path = '/content/gdrive/MyDrive/OfficeHomeDataset/OfficeHomeDataset_10072016/Clipart'

source_dataset, target_dataset, label_encoder = create_datasets(source_path, target_path)

In [ ]:
# class GradientReversalLayer(tf.keras.layers.Layer):
#     def __init__(self):
#         super(GradientReversalLayer, self).__init__()

#     def call(self, x):
#         return tf.negative(x)

#     def get_config(self):
#         config = super(GradientReversalLayer, self).get_config()
#         return config

# def create_dann_model(num_classes):
#     # Feature Extractor
#     inputs = layers.Input(shape=(224, 224, 3))
#     feature_extractor = tf.keras.applications.ResNet50(include_top=False, weights='imagenet', input_tensor=inputs)
#     features = layers.GlobalAveragePooling2D()(feature_extractor.output)
#     features = layers.Dense(256, activation='relu')(features)

#     # Label Predictor
#     label_output = layers.Dense(num_classes, activation='softmax', name='label_output')(features)

#     # Domain Discriminator
#     grl = GradientReversalLayer()(features)
#     domain_output = layers.Dense(64, activation='relu')(grl)
#     domain_output = layers.Dense(1, activation='sigmoid', name='domain_output')(domain_output)

#     model = models.Model(inputs=inputs, outputs=[label_output, domain_output])
#     return model

# # DANN 모델 생성
# num_classes = len(label_encoder.word_index)
# dann_model = create_dann_model(num_classes)

# # 손실 함수 및 옵티마이저 정의
# label_loss = tf.keras.losses.SparseCategoricalCrossentropy()
# domain_loss = tf.keras.losses.BinaryCrossentropy()
# optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)

# # 유클리드 거리 계산 함수
# def euclidean_distance(x, y):
#     return tf.reduce_sum(tf.square(x - y), axis=-1)

# # 학습 루프
# @tf.function
# def train_step(source_images, source_labels, target_images):
#     with tf.GradientTape() as tape:
#         # 소스 도메인
#         source_label_pred, source_domain_pred = dann_model(source_images, training=True)

#         # source_labels = tf.squeeze(source_labels, axis=-1)

#         source_label_loss = label_loss(source_labels, source_label_pred)
#         source_domain_loss = domain_loss(tf.ones_like(source_domain_pred), source_domain_pred)

#         # 타겟 도메인
#         target_label_pred, target_domain_pred = dann_model(target_images, training=True)
#         target_domain_loss = domain_loss(tf.zeros_like(target_domain_pred), target_domain_pred)

#         # 유클리드 거리 손실
#         distance_loss = euclidean_distance(source_label_pred, target_label_pred)

#         # 총 손실
#         total_loss = source_label_loss + source_domain_loss + target_domain_loss + 0.1 * distance_loss

#     gradients = tape.gradient(total_loss, dann_model.trainable_variables)
#     optimizer.apply_gradients(zip(gradients, dann_model.trainable_variables))

#     return total_loss

# # 학습 실행
# EPOCHS = 10
# for epoch in range(EPOCHS):
#     for (source_images, source_labels), target_images in zip(source_dataset, target_dataset):
#         loss = train_step(source_images, source_labels, target_images)
#     print(f"Epoch {epoch + 1}, Loss: {loss.numpy()}")

In [ ]:
# class GradientReversalLayer(tf.keras.layers.Layer):
#     def __init__(self):
#         super(GradientReversalLayer, self).__init__()

#     def call(self, x):
#         return tf.negative(x)

#     def get_config(self):
#         config = super(GradientReversalLayer, self).get_config()
#         return config

# def create_dann_model(num_classes):
#     # Feature Extractor
#     inputs = layers.Input(shape=(224, 224, 3))
#     feature_extractor = tf.keras.applications.ResNet50(include_top=False, weights='imagenet', input_tensor=inputs)
#     features = layers.GlobalAveragePooling2D()(feature_extractor.output)
#     features = layers.Dense(256, activation='relu')(features)

#     # Label Predictor
#     label_output = layers.Dense(num_classes, activation='softmax', name='label_output')(features)

#     # Domain Discriminator
#     grl = GradientReversalLayer()(features)
#     domain_output = layers.Dense(64, activation='relu')(grl)
#     domain_output = layers.Dense(1, activation='sigmoid', name='domain_output')(domain_output)

#     model = models.Model(inputs=inputs, outputs=[label_output, domain_output])
#     return model

# # DANN 모델 생성
# num_classes = len(label_encoder.word_index)
# dann_model = create_dann_model(num_classes)

# # 손실 함수 및 옵티마이저 정의
# label_loss = tf.keras.losses.SparseCategoricalCrossentropy()
# domain_loss = tf.keras.losses.BinaryCrossentropy()
# optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)

# # 유클리드 거리 계산 함수
# def euclidean_distance(x, y):
#     return tf.reduce_sum(tf.square(x - y), axis=-1)

# # 학습 루프
# @tf.function
# def train_step(source_images, source_labels, target_images):
#     with tf.GradientTape() as tape:
#         # 소스 도메인
#         source_label_pred, source_domain_pred = dann_model(source_images, training=True)
#         source_label_loss = label_loss(source_labels, source_label_pred)
#         source_domain_loss = domain_loss(tf.ones_like(source_domain_pred), source_domain_pred)

#         # # 타겟 도메인
#         # _, target_domain_pred = dann_model(target_images, training=True)
#         # target_domain_loss = domain_loss(tf.zeros_like(target_domain_pred), target_domain_pred)

#         # 타겟 도메인
#         target_label_pred, target_domain_pred = dann_model(target_images, training=True)
#         target_domain_loss = domain_loss(tf.zeros_like(target_domain_pred), target_domain_pred)

#         # 유클리드 거리 손실
#         distance_loss = euclidean_distance(source_label_pred, target_label_pred)

#         # 총 손실
#         total_loss = source_label_loss + source_domain_loss + target_domain_loss + 0.1 * distance_loss

#     gradients = tape.gradient(total_loss, dann_model.trainable_variables)
#     optimizer.apply_gradients(zip(gradients, dann_model.trainable_variables))

#     return total_loss

# # 학습 실행
# EPOCHS = 10
# for epoch in range(EPOCHS):
#     epoch_loss_avg = tf.keras.metrics.Mean()  # 에폭 손실 평균 계산기

#     for (source_images, source_labels), target_images in zip(source_dataset, target_dataset):
#         loss = train_step(source_images, source_labels, target_images)
#         epoch_loss_avg.update_state(loss)  # 손실 업데이트

#     print(f"Epoch {epoch + 1}, Loss: {epoch_loss_avg.result().numpy():.4f}")

In [ ]:
from tqdm import tqdm

class GradientReversalLayer(tf.keras.layers.Layer):
    def __init__(self):
        super(GradientReversalLayer, self).__init__()

    def call(self, x):
        return tf.negative(x)

    def get_config(self):
        config = super(GradientReversalLayer, self).get_config()
        return config

def create_dann_model(num_classes):
    # Feature Extractor
    inputs = layers.Input(shape=(224, 224, 3))
    feature_extractor = tf.keras.applications.ResNet50(include_top=False, weights='imagenet', input_tensor=inputs)
    features = layers.GlobalAveragePooling2D()(feature_extractor.output)
    features = layers.Dense(256, activation='relu')(features)

    # Label Predictor
    label_output = layers.Dense(num_classes, activation='softmax', name='label_output')(features)

    # Domain Discriminator
    grl = GradientReversalLayer()(features)
    domain_output = layers.Dense(64, activation='relu')(grl)
    domain_output = layers.Dense(1, activation='sigmoid', name='domain_output')(domain_output)

    model = models.Model(inputs=inputs, outputs=[label_output, domain_output])
    return model

# DANN 모델 생성
num_classes = len(label_encoder.word_index)
dann_model = create_dann_model(num_classes)

# 손실 함수 및 옵티마이저 정의
label_loss = tf.keras.losses.SparseCategoricalCrossentropy()
domain_loss = tf.keras.losses.BinaryCrossentropy()
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)

# 유클리드 거리 계산 함수
def euclidean_distance(x, y):
    return tf.reduce_sum(tf.square(x - y), axis=-1)

# 학습 루프
@tf.function
def train_step(source_images, source_labels, target_images):
    with tf.GradientTape() as tape:
        # 소스 도메인
        source_label_pred, source_domain_pred = dann_model(source_images, training=True)
        source_label_loss = label_loss(source_labels, source_label_pred)
        source_domain_loss = domain_loss(tf.ones_like(source_domain_pred), source_domain_pred)

        # 타겟 도메인
        target_label_pred, target_domain_pred = dann_model(target_images, training=True)
        target_domain_loss = domain_loss(tf.zeros_like(target_domain_pred), target_domain_pred)

        # 유클리드 거리 손실
        distance_loss = euclidean_distance(source_label_pred, target_label_pred)

        # 총 손실
        total_loss = source_label_loss + source_domain_loss + target_domain_loss + 0.1 * distance_loss

    gradients = tape.gradient(total_loss, dann_model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, dann_model.trainable_variables))

    return total_loss

# 데이터셋 크기 계산
total_batches = min(len(list(source_dataset)), len(list(target_dataset)))

# 학습 실행
EPOCHS = 10
for epoch in range(EPOCHS):
    print(f"Epoch {epoch + 1}/{EPOCHS}")
    epoch_loss_avg = tf.keras.metrics.Mean()

    # tqdm을 사용하여 진행 상황 표시
    with tqdm(total=total_batches, desc=f"Epoch {epoch + 1}/{EPOCHS}", unit="batch") as pbar:
        for batch, ((source_images, source_labels), target_images) in enumerate(zip(source_dataset, target_dataset)):
            loss = train_step(source_images, source_labels, target_images)
            epoch_loss_avg.update_state(loss)

            # 현재 배치의 손실 값 출력
            pbar.set_postfix({"Batch Loss": f"{loss.numpy():.4f}"})
            pbar.update(1)

            # 10배치마다 평균 손실 출력
            if (batch + 1) % 10 == 0:
                print(f"\nBatch {batch + 1}/{total_batches}, Average Loss: {epoch_loss_avg.result().numpy():.4f}")

    # 에폭 종료 후 평균 손실 출력
    print(f"\nEpoch {epoch + 1} completed. Average Loss: {epoch_loss_avg.result().numpy():.4f}\n")

94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 4s 0us/step


Completed one full pass through the dataset.


Completed one full pass through the dataset.


Completed one full pass through the dataset.


Completed one full pass through the dataset.


...

Completed one full pass through the dataset.


Completed one full pass through the dataset.


Completed one full pass through the dataset.


Batches:  55%|█████▌    | 42/76 [00:07<00:06,  5.65it/s]

KeyboardInterrupt: 

In [ ]:
def count_samples(domain_path):
    class_names = [d for d in os.listdir(domain_path) if os.path.isdir(os.path.join(domain_path, d))]
    count = sum(len(os.listdir(os.path.join(domain_path, class_name))) for class_name in class_names)
    return count

source_count = count_samples(source_path)
target_count = count_samples(target_path)
total_samples = min(source_count, target_count)
print(source_count, target_count, total_samples)

2427 4365 2427


In [ ]:
from tqdm import tqdm
import tensorflow as tf
from tensorflow.keras import layers, models

class GradientReversalLayer(tf.keras.layers.Layer):
    def __init__(self):
        super(GradientReversalLayer, self).__init__()

    def call(self, x):
        return tf.negative(x)

    def get_config(self):
        config = super(GradientReversalLayer, self).get_config()
        return config

def create_dann_model(num_classes):
    # Feature Extractor
    inputs = layers.Input(shape=(224, 224, 3))
    feature_extractor = tf.keras.applications.ResNet50(include_top=False, weights='imagenet', input_tensor=inputs)
    features = layers.GlobalAveragePooling2D()(feature_extractor.output)
    features = layers.Dense(256, activation='relu')(features)

    # Label Predictor
    label_output = layers.Dense(num_classes, activation='softmax', name='label_output')(features)

    # Domain Discriminator
    grl = GradientReversalLayer()(features)
    domain_output = layers.Dense(64, activation='relu')(grl)
    domain_output = layers.Dense(1, activation='sigmoid', name='domain_output')(domain_output)

    model = models.Model(inputs=inputs, outputs=[label_output, domain_output])
    return model

# DANN 모델 생성
num_classes = len(label_encoder.word_index)
dann_model = create_dann_model(num_classes)

# 손실 함수 및 옵티마이저 정의
label_loss = tf.keras.losses.SparseCategoricalCrossentropy()
domain_loss = tf.keras.losses.BinaryCrossentropy()
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)

# 유클리드 거리 계산 함수
def euclidean_distance(x, y):
    return tf.reduce_sum(tf.square(x - y), axis=-1)

# 학습 단계
@tf.function
def train_step(source_image, source_label, target_image):
    with tf.GradientTape() as tape:
        # # 소스 도메인
        # source_label_pred, source_domain_pred = dann_model(tf.expand_dims(source_image, 0), training=True)
        # source_label_loss = label_loss(tf.expand_dims(source_label, 0), source_label_pred)

        # 소스 도메인
        source_label_pred, source_domain_pred = dann_model(tf.expand_dims(source_image, 0), training=True)
        source_label_loss = label_loss(source_label, source_label_pred[0])  # [0]을 사용하여 배치 차원 제거

        # 타겟 도메인
        target_label_pred, target_domain_pred = dann_model(tf.expand_dims(target_image, 0), training=True)

        # Domain Discriminator Loss
        domain_preds = tf.concat([source_domain_pred, target_domain_pred], axis=0)
        domain_labels = tf.concat([tf.ones_like(source_domain_pred), tf.zeros_like(target_domain_pred)], axis=0)
        domain_loss_value = domain_loss(domain_labels, domain_preds)

        # 유클리드 거리 손실
        distance_loss = euclidean_distance(source_label_pred, target_label_pred)

        # 총 손실
        total_loss = source_label_loss + domain_loss_value + 0.1 * distance_loss

    gradients = tape.gradient(total_loss, dann_model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, dann_model.trainable_variables))

    return total_loss, source_label_loss, domain_loss_value, distance_loss

# 학습 실행
EPOCHS = 10
for epoch in range(EPOCHS):
    print(f"Epoch {epoch + 1}/{EPOCHS}")
    epoch_loss_avg = tf.keras.metrics.Mean()
    epoch_label_loss_avg = tf.keras.metrics.Mean()
    epoch_domain_loss_avg = tf.keras.metrics.Mean()
    epoch_distance_loss_avg = tf.keras.metrics.Mean()

    # tqdm을 사용하여 진행 상황 표시
    with tqdm(total=total_samples, desc=f"Epoch {epoch + 1}/{EPOCHS}", unit="sample") as pbar:
        for (source_image, source_label), target_image in zip(source_dataset, target_dataset):
            loss, label_loss, domain_loss, distance_loss = train_step(source_image, source_label, target_image)
            epoch_loss_avg.update_state(loss)
            epoch_label_loss_avg.update_state(label_loss)
            epoch_domain_loss_avg.update_state(domain_loss)
            epoch_distance_loss_avg.update_state(distance_loss)

            # 현재 샘플의 손실 값 출력
            pbar.set_postfix({
                "Total Loss": f"{loss.numpy().item():.4f}",
                "Label Loss": f"{label_loss.numpy().item():.4f}",
                "Domain Loss": f"{domain_loss.numpy().item():.4f}",
                "Distance Loss": f"{distance_loss.numpy().item():.4f}"
            })
            pbar.update(1)

            # 100개의 샘플마다 평균 손실 출력
            if pbar.n % 100 == 0:
                print(f"\nSample {pbar.n}/{total_samples}")
                print(f"Average Total Loss: {epoch_loss_avg.result().numpy():.4f}")
                print(f"Average Label Loss: {epoch_label_loss_avg.result().numpy():.4f}")
                print(f"Average Domain Loss: {epoch_domain_loss_avg.result().numpy():.4f}")
                print(f"Average Distance Loss: {epoch_distance_loss_avg.result().numpy():.4f}")

    # 에폭 종료 후 평균 손실 출력
    print(f"\nEpoch {epoch + 1} completed.")
    print(f"Average Total Loss: {epoch_loss_avg.result().numpy():.4f}")
    print(f"Average Label Loss: {epoch_label_loss_avg.result().numpy():.4f}")
    print(f"Average Domain Loss: {epoch_domain_loss_avg.result().numpy():.4f}")
    print(f"Average Distance Loss: {epoch_distance_loss_avg.result().numpy():.4f}\n")

94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 5s 0us/step
Epoch 1/10


Epoch 1/10:   4%|▍         | 100/2427 [02:06<29:37,  1.31sample/s, Total Loss=4.6326, Label Loss=3.9397, Domain Loss=0.6928, Distance Loss=0.0000]


Sample 100/2427
Average Total Loss: 5.2353
Average Label Loss: 4.5360
Average Domain Loss: 0.6992
Average Distance Loss: 0.0004


Epoch 1/10:   8%|▊         | 200/2427 [03:20<27:08,  1.37sample/s, Total Loss=4.9933, Label Loss=4.3000, Domain Loss=0.6933, Distance Loss=0.0000]


Sample 200/2427
Average Total Loss: 5.0825
Average Label Loss: 4.3862
Average Domain Loss: 0.6962
Average Distance Loss: 0.0002


...


Sample 2300/2427
Average Total Loss: 4.3217
Average Label Loss: 4.0255
Average Domain Loss: 0.2959
Average Distance Loss: 0.0040


Epoch 10/10:  99%|█████████▉| 2402/2427 [03:57<00:02, 10.23sample/s, Total Loss=3.9186, Label Loss=3.8884, Domain Loss=0.0296, Distance Loss=0.0069]


Sample 2400/2427
Average Total Loss: 4.3179
Average Label Loss: 4.0247
Average Domain Loss: 0.2928
Average Distance Loss: 0.0040


Epoch 10/10: 100%|██████████| 2427/2427 [04:00<00:00, 10.10sample/s, Total Loss=5.0107, Label Loss=4.4877, Domain Loss=0.5230, Distance Loss=0.0003]



Epoch 10 completed.
Average Total Loss: 4.3189
Average Label Loss: 4.0255
Average Domain Loss: 0.2930
Average Distance Loss: 0.0040



In [ ]:
# 평가 함수
def evaluate_model(model, images, labels):
    predictions, _ = model.predict(images)
    predicted_labels = np.argmax(predictions, axis=1)
    accuracy = np.mean(predicted_labels == labels)
    return accuracy

# 타겟 도메인 레이블 인코딩
target_encoded_labels = label_encoder.texts_to_sequences(target_labels)
target_encoded_labels = np.array(target_encoded_labels).flatten() - 1

# 모델 평가
target_accuracy = evaluate_model(dann_model, target_images, target_encoded_labels)
print(f"Target Domain Accuracy: {target_accuracy:.4f}")

In [ ]:
import numpy as np
from sklearn.metrics import accuracy_score, classification_report
from tqdm import tqdm

# def evaluate_model(model, dataset, label_encoder, num_classes):
#     true_labels = []
#     predicted_labels = []

#     for image, label in tqdm(dataset, desc="Evaluating"):
#         # 모델 예측
#         label_pred, _ = model(tf.expand_dims(image, 0), training=False)
#         predicted_label = tf.argmax(label_pred, axis=1).numpy()[0]

#         true_labels.append(label.numpy())
#         predicted_labels.append(predicted_label)

#     # 전체 정확도 계산
#     accuracy = accuracy_score(true_labels, predicted_labels)

#     # Get class names from label_encoder's word_index
#     # --- CHANGE START ---
#     class_names = list(label_encoder.word_index.keys())
#     # --- CHANGE END ---

#     # # 클래스별 정확도 계산
#     # class_report = classification_report(true_labels, predicted_labels,
#     #                                      target_names=label_encoder.classes_,
#     #                                      output_dict=True)

#     # 클래스별 정확도 계산
#     class_report = classification_report(true_labels, predicted_labels,
#                                          # --- CHANGE START ---
#                                          target_names=class_names,
#                                          # --- CHANGE END ---
#                                          output_dict=True)

#     return accuracy, class_report

def evaluate_model(model, dataset, label_encoder, num_classes):
    true_labels = []
    predicted_labels = []

    for image, label in tqdm(dataset, desc="Evaluating"):
        # 모델 예측
        label_pred, _ = model(tf.expand_dims(image, 0), training=False)
        predicted_label = tf.argmax(label_pred, axis=1).numpy()[0]

        true_labels.append(label.numpy())
        predicted_labels.append(predicted_label)

    # 전체 정확도 계산
    accuracy = accuracy_score(true_labels, predicted_labels)

    # Get unique class labels from true_labels
    # This ensures that target_names matches the actual classes in the data
    class_names = sorted(list(set(true_labels)))
    class_names = [str(label) for label in class_names]  # convert to strings for target_names


    # 클래스별 정확도 계산
    class_report = classification_report(true_labels, predicted_labels,
                                         target_names=class_names, # Fixed by using actual class names
                                         output_dict=True)

    return accuracy, class_report

# 타겟 도메인 데이터셋 준비 (라벨 포함)
def create_target_dataset_with_labels(target_path, label_encoder):
    return tf.data.Dataset.from_generator(
        lambda: image_generator(target_path, label_encoder),
        output_signature=(
            tf.TensorSpec(shape=(224, 224, 3), dtype=tf.float32),
            tf.TensorSpec(shape=(), dtype=tf.int32)
        )
    )

# 타겟 도메인 데이터셋 생성 (평가용)
target_dataset_with_labels = create_target_dataset_with_labels(target_path, label_encoder)

# # 모델 평가
# overall_accuracy, class_report = evaluate_model(dann_model, target_dataset_with_labels, label_encoder, num_classes)

# # 결과 출력
# print(f"Overall Accuracy: {overall_accuracy:.4f}")
# print("\nClass-wise Accuracy:")
# for class_name, metrics in class_report.items():
#     if class_name.isdigit():
#         print(f"Class {class_name} ({class_names[int(class_name)]}): {metrics['precision']:.4f}")

# 모델 평가
overall_accuracy, class_report = evaluate_model(dann_model, target_dataset_with_labels, label_encoder, num_classes)

# Get class names from class_report keys
# This ensures that target_names matches the actual classes in the data
class_names = list(class_report.keys())
class_names = [name for name in class_names if name.isdigit()]  # Filter out non-digit keys

# Update class names to start from 0 instead of 1
# The label_encoder might start indexing from 1, but the index starts from 0
# Therefore, if we just use the label_encoder, it would cause an index error for the class 0
class_names = list(label_encoder.word_index.keys())
#class_names = [name for name in range(len(class_names))]

# 결과 출력
print(f"Overall Accuracy: {overall_accuracy:.4f}")
print("\nClass-wise Accuracy:")
for class_name, metrics in class_report.items():
    if class_name.isdigit():
        class_index = int(class_name)  # Get class index as an integer
        if 0 <= class_index < len(class_names):  # Check if the class_index is in a valid range
            #print(f"Class {class_name} ({class_names[class_index]}): {metrics['precision']:.4f}")
            print(f"Class {class_name} ({class_names[class_index]}): {metrics['precision']:.4f}")
        else:
            print(f"Class {class_name} (Unknown): {metrics['precision']:.4f}")

print("\nDetailed Classification Report:")
print(classification_report(true_labels, predicted_labels, target_names=class_names))

Evaluating: 4365it [14:26,  5.04it/s]


Overall Accuracy: 0.0234

Class-wise Accuracy:
Class 0 (0): 0.0000
Class 1 (1): 0.0000
Class 3 (4): 0.0000
Class 4 (5): 0.0000
Class 5 (6): 0.0000
Class 6 (7): 0.0000
Class 7 (8): 0.0000
Class 8 (9): 0.0000
Class 9 (10): 0.0000
Class 10 (11): 0.0000
Class 11 (12): 0.0240
Class 12 (13): 0.0000
Class 13 (14): 0.0000
Class 14 (15): 0.0000
Class 15 (16): 0.0000
Class 16 (17): 0.0000
Class 17 (18): 0.0000
Class 18 (19): 0.0000
Class 19 (20): 0.0000
Class 20 (22): 0.0000
Class 22 (25): 0.0000
Class 23 (26): 0.0000
Class 25 (28): 0.0000
Class 26 (29): 0.0275
Class 27 (30): 0.0000
Class 28 (31): 0.0000
Class 29 (32): 0.0000
Class 30 (33): 0.0000
Class 31 (34): 0.0000
Class 32 (36): 0.0000
Class 33 (37): 0.0000
Class 34 (38): 0.0000
Class 36 (40): 0.0282
Class 37 (41): 0.0000
Class 38 (42): 0.0000
Class 39 (43): 0.0000
Class 40 (44): 0.0000
Class 41 (45): 0.0000
Class 42 (47): 0.0000
Class 43 (48): 0.0000
Class 44 (49): 0.0000
Class 45 (51): 0.0000
Class 47 (54): 0.0000
Class 48 (55): 0.0000
Cl

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


IndexError: list index out of range